# 🧠 SymPanICH-Net v2 — Google Colab Training

**Text-Guided Symmetry-Aware Panoptic Segmentation for ICH Detection**

This notebook trains SymPanICH-Net v2 on Google Colab with GPU acceleration.

## Prerequisites
- Google Colab Pro (for A100/V100 GPUs)
- Dataset uploaded to Google Drive

## Steps
1. Mount Google Drive
2. Clone repository
3. Install dependencies
4. Configure data path
5. Train the model
6. Evaluate & download results

## 1️⃣ GPU Check & Setup

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\n✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## 2️⃣ Mount Google Drive & Set Data Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset path in Google Drive
DATA_DIR = "/content/drive/MyDrive/FYP'26/data"

import os
if os.path.exists(DATA_DIR):
    print(f'✅ Data directory found: {DATA_DIR}')
    for item in os.listdir(DATA_DIR):
        full = os.path.join(DATA_DIR, item)
        if os.path.isdir(full):
            print(f'   📁 {item}/ ({len(os.listdir(full))} items)')
        else:
            print(f'   📄 {item}')
else:
    print(f'❌ Data directory not found: {DATA_DIR}')
    print('   Please update DATA_DIR above to match your Drive path')

## 3️⃣ Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!rm -rf /content/FYP26
!git clone https://github.com/Babu2107/FYP26.git /content/FYP26
%cd /content/FYP26

# Install dependencies
!pip install -q timm pytorch-lightning nibabel albumentations opencv-python-headless transformers peft scipy scikit-image

print('\n✅ Repository cloned and dependencies installed!')

## 4️⃣ Quick Smoke Test

Verify the model builds and runs correctly on GPU.

In [ ]:
import sys
sys.path.insert(0, '/content/FYP26')

import torch
from src.models.sympanich_net import SymPanICHNetV2

# Build model
model = SymPanICHNetV2(
    backbone_name='swinv2_tiny_window8_256',
    pretrained=True,
    use_context=False,  # 3ch for quick test
    num_queries=50,
    num_classes=6,
    num_decoder_layers=9,
)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters: {total_params / 1e6:.1f}M')
print(f'Trainable parameters: {trainable_params / 1e6:.1f}M')

# Forward pass on GPU
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

with torch.no_grad():
    dummy_img = torch.randn(1, 3, 256, 256).to(device)
    dummy_flip = torch.flip(dummy_img, dims=[3])
    outputs = model(dummy_img, dummy_flip)

print(f'\n✅ Forward pass successful on {device}!')
print(f'   pred_logits: {outputs["pred_logits"].shape}')
print(f'   pred_masks:  {outputs["pred_masks"].shape}')
print(f'   hv_maps:     {outputs["hv_maps"].shape}')

del model
torch.cuda.empty_cache()

## 5️⃣ Train the Model

Configure training and launch. Checkpoints auto-saved.

In [ ]:
# ==========================================
#  TRAINING CONFIGURATION
# ==========================================

CONFIG = {
    # Data
    'data_dir': DATA_DIR,
    'image_size': 256,
    'context_slices': 2,  # 2.5D context (set to 0 for standard 3ch)
    
    # Training
    'max_epochs': 100,
    'batch_size': 4,
    'gradient_accumulation': 4,  # Effective batch = 16
    'num_workers': 2,
    'precision': '16-mixed',     # FP16 for speed
    
    # Model
    'backbone': 'swinv2_tiny_window8_256',
    'num_queries': 50,
    'num_classes': 6,  # 5 hemorrhage types + background
    'num_decoder_layers': 9,
    
    # Optimizer
    'lr': 1e-4,
    'weight_decay': 0.05,
    
    # Checkpoints
    'checkpoint_dir': '/content/FYP26/checkpoints',
    'drive_backup_dir': "/content/drive/MyDrive/FYP'26/checkpoints",
}

print('📋 Training Configuration:')
for k, v in CONFIG.items():
    print(f'   {k}: {v}')

In [ ]:
import sys
sys.path.insert(0, '/content/FYP26')

import torch
try:
    import pytorch_lightning as pl
    from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
except ImportError:
    import lightning as pl
    from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor

from src.training.trainer import SymPanICHNetModule
from src.data.datamodule import ICHDataModule

# DataModule
datamodule = ICHDataModule(
    data_dir=CONFIG['data_dir'],
    image_size=CONFIG['image_size'],
    batch_size=CONFIG['batch_size'],
    num_workers=CONFIG['num_workers'],
    context_slices=CONFIG['context_slices'],
)

# Model
model = SymPanICHNetModule(
    backbone_name=CONFIG['backbone'],
    pretrained=True,
    num_queries=CONFIG['num_queries'],
    num_classes=CONFIG['num_classes'],
    num_decoder_layers=CONFIG['num_decoder_layers'],
    use_context=CONFIG['context_slices'] > 0,
    base_lr=CONFIG['lr'],
    weight_decay=CONFIG['weight_decay'],
    max_epochs=CONFIG['max_epochs'],
)

# Callbacks
callbacks = [
    ModelCheckpoint(
        dirpath=CONFIG['checkpoint_dir'],
        filename='sympanich-{epoch:02d}-{val/dice:.4f}',
        monitor='val/dice', mode='max', save_top_k=3, save_last=True,
    ),
    EarlyStopping(monitor='val/dice', patience=15, mode='max'),
    LearningRateMonitor(logging_interval='step'),
]

# Trainer
trainer = pl.Trainer(
    max_epochs=CONFIG['max_epochs'],
    accelerator='gpu',
    devices=1,
    precision=CONFIG['precision'],
    callbacks=callbacks,
    accumulate_grad_batches=CONFIG['gradient_accumulation'],
    gradient_clip_val=1.0,
    log_every_n_steps=10,
)

print('🚀 Starting training...')
trainer.fit(model, datamodule=datamodule)
print('\n✅ Training complete!')
print(f'Best model: {callbacks[0].best_model_path}')

## 6️⃣ Backup Checkpoints to Google Drive

In [ ]:
import shutil
import os

os.makedirs(CONFIG['drive_backup_dir'], exist_ok=True)

checkpoint_dir = CONFIG['checkpoint_dir']
if os.path.exists(checkpoint_dir):
    for f in os.listdir(checkpoint_dir):
        src = os.path.join(checkpoint_dir, f)
        dst = os.path.join(CONFIG['drive_backup_dir'], f)
        shutil.copy2(src, dst)
        print(f'  ✅ Backed up: {f}')
    print(f'\n✅ All checkpoints backed up to: {CONFIG["drive_backup_dir"]}')
else:
    print('❌ No checkpoints found')

## 7️⃣ Evaluate & Visualize Results

In [ ]:
# Test the best model
trainer.test(model, datamodule=datamodule, ckpt_path='best')

In [ ]:
# Visualize sample predictions
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from src.utils.visualization import CLASS_COLORS, CLASS_NAMES, colorize_mask, overlay_mask
from src.utils.panoptic_fusion import panoptic_fusion
from src.models.report_generator import ReportGenerator

model.eval()
model = model.to('cuda')

# Get test batch
datamodule.setup('test')
batch = next(iter(datamodule.test_dataloader()))

with torch.no_grad():
    images = batch['image'].cuda()
    images_flip = batch['image_flipped'].cuda()
    outputs = model(images, images_flip)

# Visualize each sample in the batch
B = min(images.shape[0], 4)  # Show up to 4 samples
fig, axes = plt.subplots(B, 4, figsize=(20, 5 * B))
if B == 1:
    axes = axes.reshape(1, -1)

reporter = ReportGenerator(image_size=256)

for b in range(B):
    img = images[b, :3].cpu().numpy().transpose(1, 2, 0)
    gt = batch['mask'][b].numpy()
    
    # Panoptic fusion
    fusion = panoptic_fusion(outputs['pred_logits'][b], outputs['pred_masks'][b])
    pred = fusion['semantic_map']
    pid = batch['patient_id'][b]
    sid = batch['slice_idx'][b]
    
    # Plot
    axes[b, 0].imshow(img[:,:,0], cmap='gray')
    axes[b, 0].set_title(f'CT Slice (P{pid}, S{sid})')
    axes[b, 0].axis('off')
    
    axes[b, 1].imshow(overlay_mask(img, gt, alpha=0.5))
    axes[b, 1].set_title('Ground Truth')
    axes[b, 1].axis('off')
    
    axes[b, 2].imshow(overlay_mask(img, pred, alpha=0.5))
    axes[b, 2].set_title(f'Prediction ({len(fusion["segments"])} det)')
    axes[b, 2].axis('off')
    
    axes[b, 3].imshow(colorize_mask(pred))
    axes[b, 3].set_title('Pred Mask')
    axes[b, 3].axis('off')

# Legend
patches = [mpatches.Patch(color=np.array(CLASS_COLORS[i])/255, label=CLASS_NAMES[i]) for i in range(1, 6)]
fig.legend(handles=patches, loc='lower center', ncol=5, fontsize=12)
plt.suptitle('SymPanICH-Net v2 — Predictions', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('/content/FYP26/results_visualization.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n✅ Visualization saved!')

## 8️⃣ Generate AI Clinical Report

In [ ]:
# Generate report for the last visualized prediction
segments = fusion['segments']
if segments:
    classes = np.array([s['class_id'] for s in segments])
    masks = np.stack([s['mask'] for s in segments])
    scores = np.array([s['score'] for s in segments])
    
    report = reporter.generate(
        pred_classes=classes,
        pred_masks=masks,
        pred_scores=scores,
        patient_id=f'Patient_{pid}',
    )
    print(report)
else:
    print('No hemorrhage detected in this slice.')

---
## 📝 Notes

- **Colab Pro** recommended for A100 GPU (40GB VRAM)
- **Training time**: ~4-6 hours for 100 epochs on A100
- **Checkpoints** auto-backed to Google Drive
- **For local inference**: download checkpoint and run:
  ```bash
  python scripts/visualize.py --checkpoint best.ckpt --num_samples 10
  ```